# Pixel wise attribution --> Integrated gradients

In [ ]:
#for one output
def call_model_function2(model_in_use,images,target_class_idx_list=[554]):
    target_class_idx =  target_class_idx_list[0]
    images = tf.convert_to_tensor(images)
    with tf.GradientTape() as tape:
        tape.watch(images)
        # print('starting inference')
        output_layer = model_in_use(images)
        # print('completed inference')
        output_node = output_layer[:,target_class_idx]
        # print('starting gradient calculation for the label')
        
        gradients = np.array(tape.gradient(output_node, images))

        # print('completed gradient calculation for the label')
        return  gradients

#for multiple outputs
def call_model_function(images,target_class_idx_list=[554]):
    target_class_idx =  target_class_idx_list
    images = tf.convert_to_tensor(images)
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(images)
        # print('starting inference')
        output_layer = model2(images)
        # print('completed inference')
        output_node = tf.gather(output_layer, target_class_idx, axis=1)
        # print('starting gradient calculation for the label')
        print(output_node.shape)
        gradients = []
        for i in range(len(target_class_idx)):
            gradient = np.array(tape.gradient(output_node[:,i], images))
            gradients.append(gradient)
            
        gradients=np.asarray(gradients)
        print(np.sum(gradients))

        # print('completed gradient calculation for the label')
        return  gradients

In [ ]:
def Calculate_IG_attr(model_in_use,x_value,target_class_idx_list,batch_size):
    print('started IG')
    max_min_baselines=True
    x_steps=50
    target_class_idx_list=target_class_idx_list
    if max_min_baselines is True:
        x_baselines = []
        x_baselines.append(np.min(x_value) * np.ones_like(x_value))
        x_baselines.append(np.max(x_value) * np.ones_like(x_value))
    else:
        x_baselines = []
        x_baselines.append(np.zeros_like(x_value))
        x_baselines.append(np.ones_like(x_value))
    # print(x_baselines)
    # print('completed baselining')
    grads = []
    for x_baseline in x_baselines:
        # print('starting IG for Baselinex')
        x_diff = x_value - x_baseline
        total_gradients = np.zeros_like(x_value, dtype=np.float32)
        x_step_batched = []
        for alpha in np.linspace(0, 1, x_steps):
            x_step = x_baseline + alpha * x_diff
            x_step_batched.append(x_step)
            # if len(x_step_batched) == batch_size or alpha == 1:
            if alpha == 1:
                x_step_batched = np.asarray(x_step_batched)
                # print('sending to model')
                # print(x_step_batched.shape)
                
                call_model_output = call_model_function2(model_in_use,x_step_batched,target_class_idx_list=target_class_idx_list)
        
            # self.format_and_check_call_model_output(call_model_output,
            #                                         x_step_batched.shape,
            #                                         self.expected_keys)
        
                total_gradients += call_model_output.sum(axis=0)
                x_step_batched = []
        
        grads.append(total_gradients * x_diff / x_steps)
    attr = np.mean(grads, axis=0)
    print('completed IG calculation')
    attr=attr.max(axis=-1) #change here if there is a change in shape
    print('picking up max gradients of 3 channels')
    return attr

In [ ]:
def check_pixelwise_attributions_and_load(model_in_use,topk,top10list,pth,redo=False):

    if os.path.exists(pth+'_pwa'+'.pkl') and redo==False:
        with open(pth+'_pwa'+'.pkl', 'rb') as file:
            pixelwise_attributions=pickle.load(file)
        print('loaded pixel wise attributions from file')
        return pixelwise_attributions

    else:
        print('calculating pixel wise attributions')
        pixelwise_attributions={}
        for i in range(topk):
            print('for '+str(top10list[i]))
            target_class_idx_list=[top10list[i]]
            pixelwise_attributions[top10list[i]]=Calculate_IG_attr(model_in_use,im,target_class_idx_list,batch_size=20)
        with open(pth+'_pwa'+'.pkl', 'wb') as file:
            pickle.dump(pixelwise_attributions, file)
            file.close()
        return pixelwise_attributions

        # dont save them into pkl file we can save along wirth xrai attribution along with segmentation